In [32]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeCV, Ridge, LassoCV
from scipy.stats import skew

In [42]:
train = pd.read_csv('train.csv')
# log transform target
train['SalePrice'] = np.log1p(train['SalePrice'])
test = pd.read_csv('test.csv')
data = pd.concat((train.loc[:,'MSSubClass':'SaleCondition'],test.loc[:,'MSSubClass':'SaleCondition']))
# log transform skewed numeric features
num_features = data.dtypes[data.dtypes != 'object'].index
skewed_feats = train[num_features].apply(lambda x: skew(x.dropna()))
skewed_feats = skewed_feats[skewed_feats > 0.75].index
data = pd.get_dummies(data)
data = data.fillna(data.mean())
data[skewed_feats] = np.log1p(data[skewed_feats])
Xtrain = data[:train.shape[0]].values
Xtest = data[train.shape[0]:].values
y = train.SalePrice.values

In [47]:
def rmse_cv(model):
    rmse = np.sqrt(-cross_val_score(model, Xtrain, y, scoring='neg_mean_squared_error', cv = 5))
    return(rmse)

In [53]:
model_lasso = LassoCV(alphas = [1, 0.1, 0.001, 0.0005], cv = 5).fit(Xtrain, y)
preds = np.expm1(model_lasso.predict(Xtest))
solution = pd.DataFrame({"id":test.Id, "SalePrice":preds})
solution.to_cs